### Loading Libraries

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window

### Reading CSV

#### Option 1

In [0]:
data=spark.read.csv("dbfs:/FileStore/tables/en_lpor_explorer.csv",header=True,inferSchema=True)

#### Option 2

In [0]:
data=(spark.read.format("csv")
      .option("path","dbfs:/FileStore/tables/en_lpor_explorer.csv")
      .option("header","True")
      .option("inferSchema","True")
      .load())

### Descriptive Statistics Summary

#### Basic MCT check

In [0]:
display(data.describe())

summary,School,Gender,Age,Housing_Type,Family_Size,Parental_Status,Mother_Education,Father_Education,Mother_Work,Father_Work,Reason_School_Choice,Legal_Responsibility,Commute_Time,Weekly_Study_Time,Extra_Educational_Support,Parental_Educational_Support,Private_Tutoring,Extracurricular_Activities,Attended_Daycare,Desire_Graduate_Education,Has_Internet,Is_Dating,Good_Family_Relationship,Free_Time_After_School,Time_with_Friends,Alcohol_Weekdays,Alcohol_Weekends,Health_Status,School_Absence,Grade_1st_Semester,Grade_2nd_Semester
count,649,649,649,649,649,649,649,649,649,649,649,649,649,649,649,649,649,649,649,649,649,649,649,649,649,649,649,649,649,649,649
mean,null,null,16.7442218798151,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,3.6594761171032357,11.399075500770415,11.570107858243452
stddev,null,null,1.2181376394800634,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,4.640758808397153,2.7452651284463707,2.913638664303866
min,Gabriel Pereira,Female,15,Rural,Above 3,Living Together,High School,High School,Health,Health,Course Preference,Father,15 to 30 min,2 to 5h,No,No,No,No,No,No,No,No,Excellent,High,High,High,High,Fair,0,0,0
max,Mousinho da Silveira,Male,22,Urban,Up to 3,Separated,Primary School,Primary School,other,other,Reputation,Other,Up to 15 min,Up to 2h,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Very Poor,Very Low,Very Low,Very Low,Very Low,Very Poor,32,19,19


#### Null values in column check

In [0]:
def null_values(df):
    for i in df.columns:
        if df.filter(col(i).isNull()).count()>0:
            print(i,df.filter(col(i).isNull()).count())
    print('No null values detected')
null_values(data)

No null values detected


### Exploratory Data Analysis

#### Univariate Analysis

In [0]:
#display(data)

#### Feature Engineering

In [0]:
data=data.withColumn('combined_score',col("Grade_1st_Semester")+col("Grade_2nd_Semester"))

In [0]:
data=data.drop('Grade_1st_Semester','Grade_2nd_Semester')

In [0]:
data=data.withColumn('result',when(col('combined_score')>=24,1).otherwise(0))

#### Bivariate analysis

In [0]:
#display(data)

### Encoding

In [0]:
#display(data)

In [0]:
for i in data.columns:
    data=data.withColumn(f"category_frequency_{i}", (count(i).over(Window.partitionBy(i)))/data.count())

In [0]:
#display(data)

School,Gender,Age,Housing_Type,Family_Size,Parental_Status,Mother_Education,Father_Education,Mother_Work,Father_Work,Reason_School_Choice,Legal_Responsibility,Commute_Time,Weekly_Study_Time,Extra_Educational_Support,Parental_Educational_Support,Private_Tutoring,Extracurricular_Activities,Attended_Daycare,Desire_Graduate_Education,Has_Internet,Is_Dating,Good_Family_Relationship,Free_Time_After_School,Time_with_Friends,Alcohol_Weekdays,Alcohol_Weekends,Health_Status,School_Absence,combined_score,result,category_frequency_School,category_frequency_Gender,category_frequency_Age,category_frequency_Housing_Type,category_frequency_Family_Size,category_frequency_Parental_Status,category_frequency_Mother_Education,category_frequency_Father_Education,category_frequency_Mother_Work,category_frequency_Father_Work,category_frequency_Reason_School_Choice,category_frequency_Legal_Responsibility,category_frequency_Commute_Time,category_frequency_Weekly_Study_Time,category_frequency_Extra_Educational_Support,category_frequency_Parental_Educational_Support,category_frequency_Private_Tutoring,category_frequency_Extracurricular_Activities,category_frequency_Attended_Daycare,category_frequency_Desire_Graduate_Education,category_frequency_Has_Internet,category_frequency_Is_Dating,category_frequency_Good_Family_Relationship,category_frequency_Free_Time_After_School,category_frequency_Time_with_Friends,category_frequency_Alcohol_Weekdays,category_frequency_Alcohol_Weekends,category_frequency_Health_Status,category_frequency_School_Absence,category_frequency_combined_score,category_frequency_result
Mousinho da Silveira,Male,18,Rural,Above 3,Living Together,High School,Lower Secondary School,Services,other,Course Preference,Mother,Up to 15 min,Up to 2h,No,No,No,No,Yes,No,Yes,No,Poor,Moderate,Very Low,Low,Low,Very Good,0,4,0,0.34822804314329736,0.4098613251155624,0.21571648690292758,0.3035439137134052,0.7041602465331279,0.8767334360554699,0.21417565485362094,0.3220338983050847,0.20955315870570107,0.5654853620955316,0.4391371340523883,0.7010785824345146,0.5639445300462249,0.3266563944530046,0.8952234206471494,0.386748844375963,0.9399075500770416,0.514637904468413,0.802773497688752,0.10631741140215717,0.7673343605546995,0.6317411402157165,0.04468412942989214,0.386748844375963,0.07395993836671803,0.1864406779661017,0.23112480739599384,0.38366718027734975,0.37596302003081666,0.0015408320493066256,0.559322033898305
Mousinho da Silveira,Female,19,Urban,Above 3,Living Together,Primary School,Primary School,Homemaker,Services,Other,Father,15 to 30 min,Up to 2h,No,No,No,No,Yes,No,No,No,Excellent,Very High,Very High,Low,Moderate,Poor,0,5,0,0.34822804314329736,0.5901386748844376,0.04930662557781202,0.6964560862865947,0.7041602465331279,0.8767334360554699,0.22033898305084745,0.26810477657935283,0.20801232665639446,0.27889060092449924,0.11093990755007704,0.2357473035439137,0.32819722650231126,0.3266563944530046,0.8952234206471494,0.386748844375963,0.9399075500770416,0.514637904468413,0.802773497688752,0.10631741140215717,0.23266563944530047,0.6317411402157165,0.2773497688751926,0.10477657935285054,0.1694915254237288,0.1864406779661017,0.18489984591679506,0.12018489984591679,0.37596302003081666,0.0030816640986132513,0.559322033898305
Mousinho da Silveira,Female,18,Rural,Up to 3,Separated,Higher Education,Lower Secondary School,Teacher,other,Reputation,Mother,Up to 15 min,2 to 5h,No,No,No,Yes,Yes,Yes,Yes,Yes,Excellent,Moderate,Very Low,Very Low,Very Low,Very Good,0,5,0,0.34822804314329736,0.5901386748844376,0.21571648690292758,0.3035439137134052,0.29583975346687214,0.12326656394453005,0.2696456086286595,0.3220338983050847,0.11093990755007704,0.5654853620955316,0.22033898305084745,0.7010785824345146,0.5639445300462249,0.4699537750385208,0.8952234206471494,0.386748844375963,0.9399075500770416,0.48536209553158705,0.802773497688752,0.8936825885978429,0.7673343605546995,0.3682588597842835,0.2773497688751926,0.386748844375963,0.07395993836671803,0.6949152542372882,0.3805855

#### Selecting relevant columns

In [0]:
df2=data.select('category_frequency_School',
 'category_frequency_Gender',
 'category_frequency_Age',
 'category_frequency_Housing_Type',
 'category_frequency_Family_Size',
 'category_frequency_Parental_Status',
 'category_frequency_Mother_Education',
 'category_frequency_Father_Education',
 'category_frequency_Mother_Work',
 'category_frequency_Father_Work',
 'category_frequency_Reason_School_Choice',
 'category_frequency_Legal_Responsibility',
 'category_frequency_Commute_Time',
 'category_frequency_Weekly_Study_Time',
 'category_frequency_Extra_Educational_Support',
 'category_frequency_Parental_Educational_Support',
 'category_frequency_Private_Tutoring',
 'category_frequency_Extracurricular_Activities',
 'category_frequency_Attended_Daycare',
 'category_frequency_Desire_Graduate_Education',
 'category_frequency_Has_Internet',
 'category_frequency_Is_Dating',
 'category_frequency_Good_Family_Relationship',
 'category_frequency_Free_Time_After_School',
 'category_frequency_Time_with_Friends',
 'category_frequency_Alcohol_Weekdays',
 'category_frequency_Alcohol_Weekends',
 'category_frequency_Health_Status',
 'School_Absence','result')

In [0]:
#display(df2)

#### Train test split

In [0]:
df2=df2.toPandas()

In [0]:
from sklearn.model_selection import train_test_split
X=df2.drop('result',axis=1)
y=df2['result']
xtrain,xtest,ytrain,ytest=train_test_split(X,y)

In [0]:
from sklearn.metrics import confusion_matrix,classification_report,cohen_kappa_score,accuracy_score,recall_score,f1_score,precision_score

In [0]:
!pip install xgboost

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


### Machine Learning Model

In [0]:
!pip install mlflow

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 13.7 MB/s eta 0:00:00
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import mlflow as mlflow
l=[]
for i in ['LogisticRegression','KNeighborsClassifier','GaussianNB','DecisionTreeClassifier','RandomForestClassifier','XGBClassifier']:
    with mlflow.start_run(experiment_id="1217255725030073"):
        params={'RandomForestClassifier':
                    {'n_estimators':[400,600,700,1000,1200],'max_depth':[4,5,6,7,8],'criterion':['entropy']},
                'KNeighborsClassifier':
                    {'n_neighbors': [3, 5, 7],'weights': ['uniform', 'distance'],'p': [1, 2]},
                'DecisionTreeClassifier':
                    {'criterion': ['gini', 'entropy'],'max_depth': [None, 5, 10, 15],'min_samples_split': [2, 5, 10],'min_samples_leaf': [1, 2, 4]},
                'LogisticRegression':
                    {'penalty': ['l1', 'l2', 'elasticnet', 'none'],'C': [0.001, 0.01, 0.1, 1, 10, 100],'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],'max_iter': [50, 100, 200]},
                'GaussianNB': 
                    {'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1]},
                'XGBClassifier':
                    {'learning_rate': [0.01, 0.1, 0.2],'n_estimators': [50, 100, 200],'max_depth': [3, 4, 5],'subsample': [0.8, 1.0],'colsample_bytree': [0.8, 1.0]}  
                }
                                    
        gscv=GridSearchCV(estimator=globals()[i](),param_grid=params[i],scoring='f1',cv=3,verbose=3)
        gscv.fit(xtrain,ytrain)
        model=globals()[i](**gscv.best_params_)
        model.fit(xtrain,ytrain)
        y_pred=model.predict(xtest)
        l.append({
            'model':i,
            'recall':recall_score(ytest,y_pred),
            'precision':precision_score(ytest,y_pred),
            'f1_score':f1_score(ytest,y_pred)
        })
        mlflow.log_params({
            'model':i,
            'recall':recall_score(ytest,y_pred),
            'precision':precision_score(ytest,y_pred),
            'f1_score':f1_score(ytest,y_pred)
        })

Fitting 3 folds for each of 360 candidates, totalling 1080 fits
[CV 1/3] END C=0.001, max_iter=50, penalty=l1, solver=newton-cg;, score=nan total time=   0.0s
[CV 2/3] END C=0.001, max_iter=50, penalty=l1, solver=newton-cg;, score=nan total time=   0.0s
[CV 3/3] END C=0.001, max_iter=50, penalty=l1, solver=newton-cg;, score=nan total time=   0.0s
[CV 1/3] END C=0.001, max_iter=50, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/3] END C=0.001, max_iter=50, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/3] END C=0.001, max_iter=50, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 1/3] END C=0.001, max_iter=50, penalty=l1, solver=liblinear;, score=0.000 total time=   0.0s
[CV 2/3] END C=0.001, max_iter=50, penalty=l1, solver=liblinear;, score=0.000 total time=   0.0s
[CV 3/3] END C=0.001, max_iter=50, penalty=l1, solver=liblinear;, score=0.000 total time=   0.0s
[CV 1/3] END C=0.001, max_iter=50, penalty=l1, solver=sag;, score=nan total time=   0.0

/databricks/python/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/databricks/python/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/databricks/python/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/databricks/python/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/databricks/python/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


[CV 1/3] END C=0.001, max_iter=50, penalty=none, solver=newton-cg;, score=0.584 total time=   0.1s
[CV 2/3] END C=0.001, max_iter=50, penalty=none, solver=newton-cg;, score=0.634 total time=   0.1s
[CV 3/3] END C=0.001, max_iter=50, penalty=none, solver=newton-cg;, score=0.642 total time=   0.2s
[CV 1/3] END C=0.001, max_iter=50, penalty=none, solver=lbfgs;, score=0.588 total time=   0.1s
[CV 2/3] END C=0.001, max_iter=50, penalty=none, solver=lbfgs;, score=0.634 total time=   0.1s


/databricks/python/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/databricks/python/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/databricks/python/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/databricks/python/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed

[CV 3/3] END C=0.001, max_iter=50, penalty=none, solver=lbfgs;, score=0.578 total time=   0.1s
[CV 1/3] END C=0.001, max_iter=50, penalty=none, solver=liblinear;, score=nan total time=   0.0s
[CV 2/3] END C=0.001, max_iter=50, penalty=none, solver=liblinear;, score=nan total time=   0.0s
[CV 3/3] END C=0.001, max_iter=50, penalty=none, solver=liblinear;, score=nan total time=   0.0s
[CV 1/3] END C=0.001, max_iter=50, penalty=none, solver=sag;, score=0.601 total time=   0.1s
[CV 2/3] END C=0.001, max_iter=50, penalty=none, solver=sag;, score=0.643 total time=   0.0s
[CV 3/3] END C=0.001, max_iter=50, penalty=none, solver=sag;, score=0.472 total time=   0.0s
[CV 1/3] END C=0.001, max_iter=50, penalty=none, solver=saga;, score=0.601 total time=   0.0s
[CV 2/3] END C=0.001, max_iter=50, penalty=none, solver=saga;, score=0.560 total time=   0.0s
[CV 3/3] END C=0.001, max_iter=50, penalty=none, solver=saga;, score=0.459 total time=   0.0s
[CV 1/3] END C=0.001, max_iter=100, penalty=l1, solve

...

[CV 3/3] END C=100, max_iter=200, penalty=none, solver=lbfgs;, score=0.632 total time=   0.2s
[CV 1/3] END C=100, max_iter=200, penalty=none, solver=liblinear;, score=nan total time=   0.0s
[CV 2/3] END C=100, max_iter=200, penalty=none, solver=liblinear;, score=nan total time=   0.0s
[CV 3/3] END C=100, max_iter=200, penalty=none, solver=liblinear;, score=nan total time=   0.0s
[CV 1/3] END C=100, max_iter=200, penalty=none, solver=sag;, score=0.600 total time=   0.1s
[CV 2/3] END C=100, max_iter=200, penalty=none, solver=sag;, score=0.653 total time=   0.0s
[CV 3/3] END C=100, max_iter=200, penalty=none, solver=sag;, score=0.562 total time=   0.0s
[CV 1/3] END C=100, max_iter=200, penalty=none, solver=saga;, score=0.606 total time=   0.0s


/databricks/python/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/databricks/python/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/databricks/python/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/databricks/python/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
486 fits failed out of a total of 1080.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
54 

[CV 2/3] END C=100, max_iter=200, penalty=none, solver=saga;, score=0.667 total time=   0.0s
[CV 3/3] END C=100, max_iter=200, penalty=none, solver=saga;, score=0.547 total time=   0.0s


/databricks/python/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV 1/3] END n_neighbors=3, p=1, weights=uniform;, score=0.571 total time=   0.0s
[CV 2/3] END n_neighbors=3, p=1, weights=uniform;, score=0.585 total time=   0.0s
[CV 3/3] END n_neighbors=3, p=1, weights=uniform;, score=0.618 total time=   0.0s
[CV 1/3] END n_neighbors=3, p=1, weights=distance;, score=0.571 total time=   0.0s
[CV 2/3] END n_neighbors=3, p=1, weights=distance;, score=0.577 total time=   0.0s
[CV 3/3] END n_neighbors=3, p=1, weights=distance;, score=0.614 total time=   0.0s
[CV 1/3] END n_neighbors=3, p=2, weights=uniform;, score=0.558 total time=   0.0s
[CV 2/3] END n_neighbors=3, p=2, weights=uniform;, score=0.583 total time=   0.0s
[CV 3/3] END n_neighbors=3, p=2, weights=uniform;, score=0.591 total time=   0.0s
[CV 1/3] END n_neighbors=3, p=2, weights=distance;, score=0.558 total time=   0.0s
[CV 2/3] END n_neighbors=3, p=2, weights=distance;, score=0.583 total time=   0.0s
[CV 3/3] END n_neighbors=3, p=2,

In [0]:
import pandas as pd
pd.DataFrame(l).sort_values('recall',ascending=False)

,model,recall,precision,f1_score
2,GaussianNB,0.844156,0.590909,0.695187
0,LogisticRegression,0.649351,0.617284,0.632911
1,KNeighborsClassifier,0.649351,0.568182,0.606061
5,XGBClassifier,0.636364,0.612500,0.624204
4,RandomForestClassifier,0.610390,0.635135,0.622517
3,DecisionTreeClassifier,0.506494,0.549296,0.527027
